Provincia con más número de artículos (Bokeh)

In [15]:
#Traer la primera fuente de datos

import pandas as pd
articulos = pd.read_csv("../DatosCSV/data.csv", sep=";")
articulos

,Unnamed: 0,ANO,TIPO,NOMBRE UNIVERSIDAD,TIPO FINANCIAMIENTO,PROVINCIA UNIVERSIDAD,BASE DATOS INDEXADA,NOMBRE REVISTA,NOMBRE ARTICULO,CAMPO AMPLIO,CAMPO ESPECIFICO,CAMPO DETALLADO,ALUMNOID
0,0,2020,ARTICULO,ESCUELA POLITECNICA NACIONAL,PUBLICA,PICHINCHA,SCOPUS,ASTROPHYSICAL JOURNAL,DIFFERENCE OF INTERMITTENCY BETWEEN ELECTRIC F...,"CIENCIAS NATURALES, MATEMÁTICAS Y ESTADÍSTICAS",CIENCIAS FÍSICAS,FÍSICA,914
1,1,2020,ARTICULO,ESCUELA POLITECNICA NACIONAL,PUBLICA,PICHINCHA,SCOPUS,ASTROPHYSICAL JOURNAL,ON THE EFFECTS OF UV PHOTONS/X-RAYS ON THE CHE...,"CIENCIAS NATURALES, MATEMÁTICAS Y ESTADÍSTICAS",CIENCIAS FÍSICAS,FÍSICA,4065
2,2,2020,ARTICULO,ESCUELA POLITECNICA NACIONAL,PUBLICA,PICHINCHA,SCOPUS,EARTH AND PLANETARY SCIENCE LETTERS,EFFECTS OF ASEISMIC RIDGE SUBDUCTION ON THE GE...,"CIENCIAS NATURALES, MATEMÁTICAS Y ESTADÍSTICAS",CIENCIAS FÍSICAS,CIENCIAS DE LA TIERRA,4432
3,3,2020,ARTICULO,ESCUELA POLITECNICA NACIONAL,PUBLICA,PICHINCHA,SCOPUS,EARTH AND PLANETARY SCIENCE LETTERS,RAPID LOCALIZED FLANK INFLATION AND IMPLICATIO...,"CIENCIAS NATURALES, MATEMÁTICAS Y ESTADÍSTICAS",CIENCIAS FÍSICAS,CIENCIAS DE LA TIERRA,1254
4,4,2020,ARTICULO,ESCUELA POLITECNICA NACIONAL,PUBLICA,PICHINCHA,SCOPUS,JOURNAL OF SOUND AND VIBRATION,COMBINING STRAIN AND ACCELERATION MEASUREMENTS...,"INGENIERÍA, INDUSTRÍA Y CONSTRUCCIÓN",INGENIERÍA Y PROFESIONES AFINES,MECÁNICA Y PROFESIONES AFINES A LA METALISTERÍA,4746
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50415,50415,2023,ARTICULO,UNIVERSIDAD NACIONAL DE EDUCACION 'UNAE',PUBLICA,CAÑAR,LATIN_INDEX,REVISTA SOCIAL FRONTERIZA,TENDENCIAS Y PERSPECTIVAS DE LA EDUCACIÓN INCL...,EDUCACIÓN,EDUCACIÓN,EDUCACIÓN,1061
50416,50416,2023,ARTICULO,UNIVERSIDAD NACIONAL DE EDUCACION 'UNAE',PUBLICA,CAÑAR,LATIN_INDEX,REVISTA SOCIAL FRONTERIZA,RETOS EN LA EDUCACIÓN DOCENTE PARA LA EDUCACIÓ...,EDUCACIÓN,EDUCACIÓN,EDUCACIÓN,277
50417,50417,2023,ARTICULO,UNIVERSIDAD NACIONAL DE EDUCACION 'UNAE',PUBLICA,CAÑAR,SCOPUS,ECOLOGICAL ECONOMICS,TRANSCENDING THE NATURE-SOCIETY DICHOTOMY: A D...,EDUCACIÓN,EDUCACIÓN,EDUCACIÓN,63
50418,50418,2023,ARTICULO,UNIVERSIDAD DEL RÍO,PARTICULAR AUTOFINANCIADA,GUAYAS,SCOPUS,REVIEW OF DEVELOPMENT ECONOMICS,A POSSIBLE MECHANISM FOR PARTIAL CROWDING-OUT ...,"CIENCIAS SOCIALES, PERIODISMO, INFORMACIÓN Y D...",CIENCIAS SOCIALES Y DEL COMPORTAMIENTO,ECONOMÍA,2749


In [16]:
#Traer la segunda fuente de datos

import sqlite3

conn = sqlite3.connect("../DatosSQL/students.db")
alumnos = pd.read_sql("select * from personal_data", con=conn)
conn.close()
alumnos

,ALUMNOID,NOMBRE,CORREO
0,1,Itziar Torrens,itorrens@outlook.com
1,2,Patricia Durán,pduran@soho.com
2,3,Gisela Bonet,gbonet@yahoo.com
3,4,Sabina Feliu,sfeliu@gmail.com
4,5,Visitación Martin,vmartin@soho.com
...,...,...,...
49995,49996,Eladio Morales,emorales@outlook.com
49996,49997,Natividad Rodriguez,nrodriguez@hotmail.com
49997,49998,Amador Rovira,arovira@hotmail.com
49998,49999,María Luisa Morante,mmorante@yahoo.com


In [18]:
datos = pd.merge(articulos, alumnos, on="ALUMNOID")

In [ ]:
!pip install bokeh

In [22]:
# importar las librerias
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show, output_file
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [23]:
#Agrupar por provincia y año, contando artículos
conteo = datos.groupby(['PROVINCIA UNIVERSIDAD', 'ANO']).size().reset_index(name='cantidad')

In [24]:
#Preparar los pares (provincia, año) como claves para el eje X
conteo['provincia'] = conteo['PROVINCIA UNIVERSIDAD'].astype(str)
conteo['ano'] = conteo['ANO'].astype(str)
conteo['x'] = list(zip(conteo['provincia'], conteo['ano']))

In [25]:
#Crear fuente de datos para Bokeh
source = ColumnDataSource(conteo)

In [26]:
# Preparar el rango del eje X
provincias = sorted(conteo['provincia'].unique())
anos = sorted(conteo['ano'].unique())
x_range = [(prov, ano) for prov in provincias for ano in anos]

In [27]:
#Crear la figura
p = figure(
    x_range=FactorRange(*x_range),
    height=500,
    width=1200,
    title="Número de artículos por provincia y año",
    toolbar_location=None,
    tools="hover",
    tooltips="@x: @cantidad"
)

In [28]:
#Dibujar las barras
p.vbar(
    x='x',
    top='cantidad',
    width=0.8,
    source=source,
    fill_color=factor_cmap('x', palette=Category20[20], factors=anos, start=1, end=2),
    line_color='white'
)

GlyphRenderer(id='p1085', ...)

In [29]:
# Dar estilos del gráfico
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.5
p.xaxis.axis_label = "Provincia / Año"
p.yaxis.axis_label = "Cantidad de artículos"
p.xgrid.grid_line_color = None

In [30]:
#Mostrar el gráfico
show(p)

Box(children=(HTML(value='\n<div id="ifr-pyg-000635954c3016e4TlIf79BUiCtHRo0E" style="height: auto">\n    <hea…